In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load data file
df = pd.read_csv('IntervalIDBase14Intersections.txt', delimiter=';')

In [3]:
# data cleaning
num_df = df.iloc[:,0:65].copy().replace('()',0).replace(to_replace=r'^(.*)$', value=1, regex=True)

df_all_edge_detection = pd.DataFrame(np.zeros((df.index[0], 32)))
for i in range(32):
    df_all_edge_detection.iloc[:,i] = num_df.iloc[:,2*i+1] + num_df.iloc[:,2*i+2]

df_traffic_light = pd.DataFrame(np.zeros((df.index[0], 8)))
for i in range(65,73):
    temp = df.iloc[:,i]/2    
    df_traffic_light.iloc[:,i-65] = temp.astype(int)
    
df_intersection_vehnum = pd.DataFrame(np.zeros((df.index[0], 16)))
for i in range(73,89):   
    df_intersection_vehnum.iloc[:,i-73] = df.iloc[:,i]
    
df_outIntersection_vehnum = pd.DataFrame(np.zeros((df.index[0], 16)))
for i in range(89,105):   
    df_outIntersection_vehnum.iloc[:,i-89] = df.iloc[:,i]
    
df_intersection_crowd_size = pd.DataFrame(np.zeros((df.index[0], 8)))
for i in range(8):
    df_intersection_crowd_size.iloc[:,i] = df.iloc[:,i+105]

In [10]:
# caculate the interval between vehicle crowd, all intersections  --V2.0

nums = range(0,32)
intervalList = []
minCrowdSize = 3
maxTimeIntervalInCrowd = 3
crowdSizeList = []

for num in nums:
    interval = 0
    firstVehicleInterval = 0
    detectorInterval = []
    currentTimeIntervalInCrowd = 0
    currentCrowdSize = 0
    crowdFlag = 0
    firstVehicleFlag = 0
    currentCrowdSizeList = []
    for key, value in df_all_edge_detection.iloc[:,num].iteritems(): 
        interval = interval + 1
        if value == 0:
            # not a crowd, just one or two vehicle
            if firstVehicleFlag == 1 and interval - firstVehicleInterval >= maxTimeIntervalInCrowd:
                currentCrowdSize = 0
                firstVehicleFlag = 0
                firstVehicleInterval = 0                        
            
            if interval == maxTimeIntervalInCrowd and crowdFlag == 1:
                currentCrowdSizeList.append(currentCrowdSize)
                crowdFlag = 0
                currentCrowdSize = 0

        else:
            currentCrowdSize = currentCrowdSize + 1        
            if currentCrowdSize >= minCrowdSize:            
                if crowdFlag == 0:
                    detectorInterval.append(firstVehicleInterval)
                    firstVehicleInterval = 0
                    firstVehicleFlag = 0
                crowdFlag = 1
                interval = 0
            elif currentCrowdSize == 1:
                if crowdFlag == 0:
                    firstVehicleInterval = interval
                    firstVehicleFlag = 1
            else:
                # if current 1< veh# < 3, interval still increase
                continue
    
    if crowdFlag == 1:
        currentCrowdSizeList.append(currentCrowdSize)
    
    intervalList.append(detectorInterval)
    crowdSizeList.append(currentCrowdSizeList)

In [16]:
for i in range(32):
    if not len(intervalList[i]) == len(crowdSizeList[i]):
        print(i)

In [18]:
# the following is to caculate the link delay
# up and down link
detectorDownNameList = ["e1Detector_E12_0_0","e1Detector_E12_1_1","e1Detector_E23_0_4","e1Detector_E23_1_5",
                        "e1Detector_E34_0_8","e1Detector_E34_1_9","e1Detector_E45_0_12","e1Detector_E45_1_13",
                        "e1Detector_E56_0_16","e1Detector_E56_1_17","e1Detector_E67_0_20","e1Detector_E67_1_21",
                        "e1Detector_E78_0_24","e1Detector_E78_1_25","e1Detector_E89_0_28","e1Detector_E89_1_29"]
detectorUpNameList = ["e1Detector_E09_1_30","e1Detector_E09_0_31","e1Detector_E98_1_26","e1Detector_E98_0_27",
                      "e1Detector_E87_1_22","e1Detector_E87_0_23","e1Detector_E76_1_18","e1Detector_E76_0_19",
                      "e1Detector_E65_1_14","e1Detector_E65_0_15","e1Detector_E54_1_10","e1Detector_E54_0_11",
                      "e1Detector_E43_1_6","e1Detector_E43_0_7","e1Detector_E32_1_2","e1Detector_E32_0_3"]
detectorOutDownNameList = ["out_e1Detector_E23_0_4","out_e1Detector_E23_1_5","out_e1Detector_E34_0_8","out_e1Detector_E34_1_9",
                           "out_e1Detector_E45_0_12","out_e1Detector_E45_1_13","out_e1Detector_E56_0_16","out_e1Detector_E56_1_17",
                           "out_e1Detector_E67_0_20","out_e1Detector_E67_1_21","out_e1Detector_E78_0_24","out_e1Detector_E78_1_25",
                           "out_e1Detector_E89_0_28","out_e1Detector_E89_1_29","out_e1Detector_E90_1_30","out_e1Detector_E90_0_31"]
detectorOutUpNameList = ["out_e1Detector_E98_1_26","out_e1Detector_E98_0_27","out_e1Detector_E87_1_22","out_e1Detector_E87_0_23",
                         "out_e1Detector_E76_1_18","out_e1Detector_E76_0_19","out_e1Detector_E65_1_14","out_e1Detector_E65_0_15",
                         "out_e1Detector_E54_1_10","out_e1Detector_E54_0_11","out_e1Detector_E43_1_6","out_e1Detector_E43_0_7",
                         "out_e1Detector_E32_1_2","out_e1Detector_E32_0_3","out_e1Detector_E21_0_0","out_e1Detector_E21_1_1"]

carIDFile = open('carIDFile14Intersections.txt', "r")
for line in carIDFile:
    carIDList = line.split(",")
carIDFile.close() 

carDownIDList = []
carUpIDList = []
for carID in carIDList:
    if carID[0] == "u":
        carUpIDList.append(carID)
    else:
        carDownIDList.append(carID)

In [20]:
df_ID_DownEdge_Time = pd.DataFrame(columns =  ["carID", "DownEdge", "Timestamp"])
i = 0
for carID in carDownIDList:
    for edge in detectorDownNameList:
        record = df.loc[df[edge].str.contains("('"+carID+"',)")]
        if not record.empty:
            df_ID_DownEdge_Time.loc[i] = [carID,edge[11:14],record.index[0]]
            i = i + 1

C:\Users\byang12\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


In [21]:
df_ID_UpEdge_Time = pd.DataFrame(columns =  ["carID", "UpEdge", "Timestamp"])
i = 0
for carID in carUpIDList:
    for edge in detectorUpNameList:
        record = df.loc[df[edge].str.contains("('"+carID+"',)")]
        if not record.empty:
            df_ID_UpEdge_Time.loc[i] = [carID,edge[11:14],record.index[0]]
            i = i + 1

C:\Users\byang12\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


In [22]:
df_ID_DownOutEdge_Time = pd.DataFrame(columns =  ["carID", "DownEdge", "Timestamp"])
i = 0
for carID in carDownIDList:
    for edge in detectorOutDownNameList:
        record = df.loc[df[edge].str.contains("('"+carID+"',)")]
        if not record.empty:
            df_ID_DownOutEdge_Time.loc[i] = [carID,edge[15:18],record.index[0]]
            i = i + 1

C:\Users\byang12\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


In [23]:
df_ID_UpOutEdge_Time = pd.DataFrame(columns =  ["carID", "UpEdge", "Timestamp"])
i = 0
for carID in carUpIDList:
    for edge in detectorOutUpNameList:
        record = df.loc[df[edge].str.contains("('"+carID+"',)")]
        if not record.empty:
            df_ID_UpOutEdge_Time.loc[i] = [carID,edge[15:18],record.index[0]]
            i = i + 1

C:\Users\byang12\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


In [25]:
df_ID_DownEdge_delay = pd.DataFrame(columns =  ["carID", "DownEdge", "delay"])
i = 0
for index, row in df_ID_DownEdge_Time.iterrows():
    if index%8 == 0:
        upStreamEdgeTimestamp = row['Timestamp']
    else:        
        downStreamEdgeTimestamp = row['Timestamp']
        df_ID_DownEdge_delay.loc[i] = [row['carID'],row['DownEdge'],downStreamEdgeTimestamp-upStreamEdgeTimestamp]
        i = i + 1
        upStreamEdgeTimestamp = row['Timestamp']

In [26]:
df_ID_UpEdge_delay = pd.DataFrame(columns =  ["carID", "UpEdge", "delay"])
i = 0
for index, row in df_ID_UpEdge_Time.iterrows():
    if index%8 == 0:
        upStreamEdgeTimestamp = row['Timestamp']
    else:        
        downStreamEdgeTimestamp = row['Timestamp']
        df_ID_UpEdge_delay.loc[i] = [row['carID'],row['UpEdge'],downStreamEdgeTimestamp-upStreamEdgeTimestamp]
        i = i + 1
        upStreamEdgeTimestamp = row['Timestamp']

In [27]:
df_ID_DownOutEdge_delay = pd.DataFrame(columns =  ["carID", "DownEdge", "delay"])
i = 0
for index, row in df_ID_DownOutEdge_Time.iterrows():
    if index%8 == 0:
        upStreamEdgeTimestamp = row['Timestamp']
    else:
        downStreamEdgeTimestamp = row['Timestamp']
        df_ID_DownOutEdge_delay.loc[i] = [row['carID'],row['DownEdge'],downStreamEdgeTimestamp-upStreamEdgeTimestamp]
        i = i + 1
        upStreamEdgeTimestamp = row['Timestamp']

In [28]:
df_ID_UpOutEdge_delay = pd.DataFrame(columns =  ["carID", "UpEdge", "delay"])
i = 0
for index, row in df_ID_UpOutEdge_Time.iterrows():
    if index%8 == 0:
        upStreamEdgeTimestamp = row['Timestamp']
    else:        
        downStreamEdgeTimestamp = row['Timestamp']
        df_ID_UpOutEdge_delay.loc[i] = [row['carID'],row['UpEdge'],downStreamEdgeTimestamp-upStreamEdgeTimestamp]
        i = i + 1
        upStreamEdgeTimestamp = row['Timestamp']

In [ ]:
# caculate the interval between vehicle crowd, I5(detc 6 7) I6(detc 8 9)
detectorNums = [7,8]  # 7:upLink, 8:downLink
#intervalList = []
DelayList = []
minCrowdSize = 3
maxTimeIntervalInCrowd = 3
for detectorNum in detectorNums:
    interval = 0
    #detectorInterval = []
    intersectionDelayList = []
    crowdDelayList = []
    currentTimeIntervalInCrowd = 0
    currentCrowdSize = 0
    crowdFlag = 0
    for key, value in df_all_edge_detection.iloc[:,detectorNum].iteritems(): 
        if value == 0:
            interval = interval + 1
            if interval > maxTimeIntervalInCrowd:
                # caculate current crowd Delay, average of all vehicle
                if crowdFlag == 1:
                    intersectionDelayList.append(int(np.mean(crowdDelayList)))
                    crowdFlag = 0
                    currentCrowdSize = 0
                    crowdDelayList = []
                else:
                    continue                    
        else:
            currentCrowdSize = currentCrowdSize + 1
            
            if detectorNum == 8:
                for i in range(2):
                    name = detectorDownNameList[2+i]
                    if not df2[name][key] == '()':
                        vehicleID = df2[name][key][2:-3]
                        currentDelay = int(df_ID_DownEdge_delay['delay'][df_ID_DownEdge_delay['carID'] == vehicleID])
                        crowdDelayList.append(currentDelay)
            if detectorNum == 7:
                for i in range(2):
                    name = detectorUpNameList[2+i]
                    if not df2[name][key] == '()':
                        vehicleID = df2[name][key][2:-3]
                        currentDelay = int(df_ID_UpEdge_delay['delay'][df_ID_UpEdge_delay['carID'] == vehicleID])
                        crowdDelayList.append(currentDelay)             
            
            if currentCrowdSize == minCrowdSize:
                if crowdFlag == 0:
                    detectorInterval.append(interval)
                crowdFlag = 1
                interval = 0
            elif currentCrowdSize > minCrowdSize:
                crowdFlag = 1
                interval = 0
            else: # if current veh# < 3, interval still increase                
                interval = interval + 1
    #intervalList.append(detectorInterval)
    DelayList.append(intersectionDelayList)